In [1]:
%run -n main.py
USER = 'alexanderkuk'
PASS = get_pass()
AUTH = (USER, PASS)

········


In [2]:
q = gh_q(
    '"from yargy" OR "import yargy" OR "from natasha" OR "import natasha"',
    extensions=('py', 'ipynb'),
)
url = gh_search_code_url(q)
response, data = call_gh(url, AUTH, TEXT_MATCH)
total = parse_serp_total(data)
print('Total:', total)

Total: 395


In [3]:
%%time
serp = []
pages = list(get_pages(total))
while pages:
    page = pages.pop(0)
    print('Page #:', page, 'To do:', len(pages))
    url = gh_search_code_url(q, page=page)
    response, data = call_gh(url, AUTH, headers=TEXT_MATCH)
    if is_broken(data):
        print('Ban')
        pages.insert(0, page)
        sleep(60)
    else:
        serp.extend(parse_serp(data))
        sleep(5)

Page #: 1 To do: 13
Page #: 2 To do: 12
Page #: 3 To do: 11
Page #: 4 To do: 10
Page #: 5 To do: 9
Ban
Page #: 5 To do: 9
Page #: 6 To do: 8
Page #: 7 To do: 7
Page #: 8 To do: 6
Page #: 9 To do: 5
Page #: 10 To do: 4
Page #: 11 To do: 3
Page #: 12 To do: 2
Page #: 13 To do: 1
Page #: 14 To do: 0
Ban
Page #: 14 To do: 0
CPU times: user 446 ms, sys: 55.3 ms, total: 501 ms
Wall time: 3min 30s


In [4]:
cache = set(load_lines(URLS))
urls = {get_serp_record_url(_) for _ in serp}
print('In cache:', len(cache & urls), 'New:', len(urls - cache))

In cache: 395 New: 0


In [5]:
pattern = '(yargy|natasha)'
for record in serp:
    url = get_serp_record_url(record)
    if url in cache:
        continue
    print(url)
    for text in record.matches:
        spans = get_spans(text, pattern)
        show_markup(text, spans)

In [6]:
dump_lines(sorted(cache | urls), URLS)